
Last updated: Mon 27 Oct (helmitaulu stuff at the end)

### On problem 1

The problem is difficult, in that there may not be a readily available recipe for it. But it's a real-life problem, and everyone should expolore it enough to see that it's not that easy! Feel free to help each other, and after the deadline (maybe already during the weekend) we'll check:

- what you did
- what the students did last year
- what we did with Mika (before any student work).





### On problem 2

Real-life time series data! There's a story behind it. All the Jamk employees need to report their worktimes into a shitty system called reportronic. Before getting the lecturer position, I worked as a part-time teacher, and it was okay for me to robot-report the daily 4h48min of work and 3h12 min of break between 8-16 (every day the same & no explanations), no matter what the reality was.

However, after I got the lecturer position (60% - 70%) in fall 2022, they started complaining: One has to explain each entry & it's not allowed to use a robot. So I refined my robot to report random entries according to predefined tasks (explanations) and sums of working hours.

That worked fine for a while, but people already knew about it, so soon the supervisors started complaining: You cannot use a robot, but you have to be completely honest.

So then (spring 2023, see [worktimes](https://student.labranet.jamk.fi/~varpha/worktimes)) I started to be honest... But my yearly working hours immediately went over the predefined limit, which resulted first in a lot of useless fiddling and a waste of time for many people, and finally in me reporting zero hours in the official system (which was okay for the supervisors, although it was not honest). I was not surprised at all!

Nowadays I use a combination of my old robot and this data shared to you. So I don't report everything in the official system, but the supervisors (and whoever else may be interested) can find the honest worktimes from the website (not updated anymore in fall 2025 as I don't have the time for that).





In [ ]:
# problem 3 related cell

import pandas as pd
from seaborn import load_dataset
import matplotlib.pyplot as plt

df = load_dataset('dowjones')
df['Date'] = pd.to_datetime(df['Date'])
# df = df.set_index('Date')
df.set_index('Date', inplace=True)

df['moving_avg'] = df['Price'].rolling(20, center=True).mean()
# df['moving_avg'] = df['Price'].expanding().mean()
# df['moving_avg'] = df['Price'].ewm(alpha=0.1, adjust=False).mean()

# df.info()
df.plot()

### A general "ml model fitting" skeleton

The rest of the course (starting from the weekend) will be about fitting machine learning models to data.

An `autofit` library doesn't exist in reality, so you need to replace that.

In [ ]:
# an ml model fitting skeleton
# autofit doesn't exist in reality

import pandas as pd
from sklearn.model_selection import train_test_split

from autofit_library import autofit
# replace this

df = pd.DataFrame()
# your data

X = df['features'] # "supervised" refers to this
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = autofit(X_train, y_train)
# fit model

model.save('my_model')
# save model

#### then later ####

model = load_model('my_model')
# load the saved model

unseen_data = pd.DataFrame()
# get some new data

X = unseen_data['features']
my_target_prediction = model.predict(X)
# make a prediction

print(my_target_prediction)
# satisfy your customer

The skeleton modified to a linear regression fit:

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
model = LinearRegression()

df = pd.read_excel('/home/varpha/dan/public/exrc_04/data/weekend/shoesize_length_gender.xlsx')

df.columns=['shoesize', 'length', 'gender', 'a', 'b']
df = df.drop(columns=['a','b'])

X = df[['length','gender']]
y = df['shoesize']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state=300)

model.fit(X_train, y_train) # fit

model.predict(X_test)
print('test score:', model.score(X_test, y_test))
# the thing above is called validation


# model.save('my_model')
# save model

#### then later ####

# model = load_model('my_model')
# load the saved model

unseen_data = pd.DataFrame.from_dict(
    {
        'length': [213],
        'gender':[1]
    }
)
# lauri markkanen

my_target_prediction = model.predict(unseen_data)
# make a prediction

print(my_target_prediction)
# satisfy your customer

## Time series intro

  - [This](https://medium.com/towards-data-science/12-things-you-should-know-about-time-series-975a185f4eb2) is an okay blogtext (more or less)
  - Note: a multiplicative data can be transformed into an additive data by taking (any) logarithm, because $$ \log(ab)=\log(a)+\log(b).$$ In the end one deals with everything in the additive fashion, and I guess the libraries nowadays can do that automatically, so one doesn't need to worry that much about the data being multiplicative. (I think.)
  - From now on we will refer to the [jakevdp book](https://jakevdp.github.io/PythonDataScienceHandbook) instead of the wesmckinney book (although neither book will help us in what follows below).

## Champagne sales stuff

Below follows a modification of the [champagne blog](https://machinelearningmastery.com/time-series-forecast-study-python-monthly-sales-french-champagne).

See also:
- [the full time series book by the same author](https://github.com/linh22/Data-Scientist-Books/blob/main/Introduction%20to%20Time%20Series%20Forecasting%20with%20Python%20How%20to%20Prepare%20Data%20and%20Develop%20Models%20to%20Predict%20the%20Future%20by%20Jason%20Brownlee%20(z-lib.org).pdf)
- [acf pacf gentle introduction](https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/) (missing from the book)
- [another explanation of the same champagne thing](https://github.com/saksham219/Time-series-forecasting---Champagne-sales/blob/master/champagne.ipynb)
- [yet another](https://github.com/ManojKumarMaruthi/Time-Series-Forecasting/tree/master)
- [and another](https://github.com/DataForScience/Timeseries)

The last link above is more general than the champagne thing. It's hard to read but it has good structure. One can probably read it with the help of AI.

## Additions (weekend)

ARIMA = AutoRegressive Integrated Moving Average

- one could do moving average only (explained during the weekend)
- autoregressive: the time series correlates with "its own past"
    - typically that doesn't happen
    - but it happens more often to the **differenced** time series (see library cell below)
- integrated: refers to the differencing operation
- one takes a weighted average of the autoregressive and moving average parts


### Part 1/7
- library cell

In [ ]:
# champagne sales example stripped & modified - part 1/7

# just a library cell
# these imports and functions are repeatedly used in
# the cells that follow

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

# create a differenced dataset that (hopefully) has
# better stationarity properties than the original
# (default differencing interval size = 1)
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return diff


# get original value back from differenced (y_hat), given history
def inverse_difference(history, y_hat, interval=1):
    return y_hat + history[-interval]

# demo
a = 2*np.array([1,2,3,4,5,6])

print(difference(a))
# this is pretty straighforward

print(inverse_difference(a[:-1], difference(a)[-1]))
# get current (here the last) value back from previous values and the current value of the differenced data


### Part 2/7
- load data
- describe data
- split to train and test parts

In [ ]:
# champagne sales example stripped & modified - part 2/7

import pandas as pd
df = pd.read_csv('/home/varpha/dan/public/exrc_04/data/weekend/monthly_champagne_sales.csv', header=0, index_col=0, parse_dates=True).squeeze('columns')

print(df.describe())

split_point = len(df) - 12
# manual splitting, the last year is for "validation"
# quotes because the last year is saved for the very end
# the "usual" validation is done before that (by splitting the train data)

dataset, validation = df[0:split_point], df[split_point:]
# train and test data

print('Dataset %d, Validation %d' % (len(dataset), len(validation)))
# print sizes

dataset.to_csv('./data/dataset.csv', header=False)
validation.to_csv('./data/validation.csv', header=False)
# save the train and test data


### Part 3/7
- apply a differencing operation to the data
- test that the differenced data is stationary

In [ ]:
# champagne sales example stripped & modified - part 3/7

# produce a differenced series (the I letter in ARIMA)
# where the differencing parameter d=12 (12 months)
# the differenced series is stationary (as tested here)

from statsmodels.tsa.stattools import adfuller

df = pd.read_csv('./data/dataset.csv', header=None, index_col=0, parse_dates=True).squeeze('columns')
X = df.values.astype('float32')
# load training dataset and save only the values (not dataframe) to X

months_in_year = 12
differenced_data = pd.Series(difference(X, months_in_year))
# use the differencing function from the library cell

differenced_data.index = df.index[months_in_year:]

adfuller_result = adfuller(differenced_data)
# test for stationarity

print('ADF Statistic: %f' % adfuller_result[0])
print('p-value: %f' % adfuller_result[1])
print('Critical Values:')
for key, value in adfuller_result[4].items():
    print('\t%s: %.3f' % (key, value))

# one does not need to save the stationary.csv as in the blog
# (it never gets used)

# okay we can plot this but the picture alone
# doesn't reveal stationarity very obviously
# (hard to detect stationarity from a picture only)
differenced_data.plot()
plt.show()



## Understanding P-Values
(An excerpt from *Essential Math for Data Science*)

When we say something is statistically significant, what do we mean by that? We hear it used loosely and frequently but what does it mean mathematically? Technically, it has to do with something called the p-value, which is a hard concept for many folks to grasp. But I think the concept of p-values makes more sense when you trace it back to its invention. While this is an imperfect example, it gets across some big ideas.

In 1925, mathematician Ronald Fisher was at a party. One of his colleagues Muriel Bristol claimed she could detect when tea was poured before milk simply by tasting it.

Intrigued by the claim, Ronald set up an experiment on the spot.

He prepared eight cups of tea. Four had milk poured first; the other four had tea poured first. He then presented them to his connoisseur colleague and asked her to identify the pour order for each. Remarkably, she identified them all correctly, and the probability of this happening by chance is 1 in 70, or 0.01428571.

This 1.4% probability is what we call the p-value, the probability of something occurring by chance rather than because of a hypothesized explanation. Without going down a rabbit hole of combinatorial math, the probability that Muriel completely guessed the cups correctly is 1.4%. What exactly does that tell you?

When we frame an experiment, whether it is determining it organic donuts cause weight gain or living near power lines causes cancer, we always have to entertain the possibility that random luck played a role. Just like there is a 1.4% chance Muriel identified the cups of tea correctly simply by guessing, there's always a chance randomness just gave us a good hand like a slot machine. This helps us frame our null hypothesis (Ho), saying that the variable in question had no impact on the experiment and any positive results are just random luck. The alternative hypothesis (H) poses that a variable in question (called the controlled variable) is causing a positive result.

Traditionally, the threshold for statistical significance is a p-value of 5% or less, or .05. Since .014 is less than .05, this would mean we can reject our null hypothesis that Muriel was randomly guessing. We can then promote the alternative hypothesis that Muriel had a special ability to detect whether tea or milk was poured first.


### Part 4/7 (version a &mdash; not recommended)
- find optimal ARIMA parameters by brute force

In [ ]:
# champagne sales example stripped & modified - part 4/7 version a

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

import warnings

# evaluate an ARIMA model for a given order (p,d,q) and return RMSE
# the same thing is done faster in version b (next cell)

def evaluate_arima_model(X, arima_order):

    # (float32 not redundant because not the same X as in previous cell)
    X = X.astype('float32')

    # here is done another split
    # this is the "typical" model fit split
    # that is called validation
    train_size = int(len(X) * 0.50)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]

    # make predictions
    predictions = list()
    for t in range(len(test)):
        # difference data
        months_in_year = 12
        diff = np.array(difference(history, months_in_year))
        model = ARIMA(diff, order=arima_order)
        model_fit = model.fit()
        y_hat = model_fit.forecast()[0]
        y_hat = inverse_difference(history, y_hat, months_in_year)
        predictions.append(y_hat)
        history.append(test[t])
    # calculate score (root mean square error)
    # we get back to that in round 5
    # (meanwhile one can think: score = correlation coefficient for linear fit)
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
    	for d in d_values:
    		for q in q_values:
    			order = (p,d,q)
    			try:
    				rmse = evaluate_arima_model(dataset, order)
    				if rmse < best_score:
    					best_score, best_cfg = rmse, order
    				print('ARIMA%s RMSE=%.3f' % (order,rmse))
    			except:
    				continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

# load dataset
df = pd.read_csv('./data/dataset.csv', header=None, index_col=0, parse_dates=True).squeeze('columns')

# evaluate parameters
p_values = range(0, 7)
d_values = range(0, 3)
q_values = range(0, 7)
warnings.filterwarnings("ignore")
evaluate_models(df.values, p_values, d_values, q_values)


### Part 4/7 (version b &mdash; recommended)
- find optimal ARIMA parameters (= best fit) using [pmdarima](https://github.com/alkaline-ml/pmdarima)
- see also [sktime](https://github.com/sktime/sktime)

In [ ]:
# champagne sales example stripped & modified - part 4/7 version b

import pmdarima as pm
from pmdarima.arima import auto_arima

from pandas import read_csv

def evaluate_models(dataset):
    
    train, test = pm.model_selection.train_test_split(dataset, train_size=90)
    # pmdarima has train_test_split too
    # note: this split is used to find the best fit in
    # the dataset (the split done at the beginning is different)
    
    # arima_model = auto_arima(train)
    # one can try brainless first (line above)
    # but the below works better
    
    arima_model = auto_arima(
        train,
        start_p=0,
        d=1,
        start_q=0, 
        max_p=5,
        max_d=5,
        max_q=5,
        start_P=0,
        D=1,
        start_Q=0,
        max_P=5,
       max_D=5,
       max_Q=5,
       m=12, # found in the stationarity test before
       seasonal=True,
       error_action='warn',
       trace = True,
       supress_warnings=True,
       stepwise = True,
       random_state=20,
       n_fits = 50
    )

    return arima_model


##### main #####

df = read_csv('./data/dataset.csv', header=None, index_col=0, parse_dates=True).squeeze('columns')
model = evaluate_models(df.values)

print(model.summary())
# you should get [12] (because yearly seasonality)
# please use the latter triple in the next stage


### Part 5/7
- validate

In [ ]:
# champagne sales example stripped & modified - part 5/7

# in this cell one just validates
# (similar to linear regression score)
# (but now the validation process and score concept are more complicated)

# check gaussianity of the residuals
# a common thing to do
# ideally gaussian with zero mean
# if mean not zero then it has to be subtracted from the predictions
# (the predictions are biased by the mean)

# this cell doesn't save anything
# it just checks the gaussianity
# and produces a mean (= bias)

# input your best parameters (latter triple, see below) from the previous step (4/7)
# and save the mean as the bias in the next step (6/7)

# Best model:  ARIMA(0,1,2)(2,1,0)[12]     
# (copy-pasted from the output of the previous cell)
# the latter triple is the one we need here

from statsmodels.tsa.arima.model import ARIMA

df = pd.read_csv('./data/dataset.csv', header=None, index_col=0, parse_dates=True).squeeze('columns')

X = df.values.astype('float32')

# validation split 50-50 (at first -- will be "walked forward")
train_size = int(len(X) * 0.50)
train, test = X[0:train_size], X[train_size:]

# walk-forward validation
history = [x for x in train]
predictions = list()
for i in range(len(test)):
    months_in_year = 12
    diff = list(difference(history, months_in_year))
    model = ARIMA(diff, order=(2,1,0))
    # (ARIMA is from https://www.statsmodels.org/devel/generated/statsmodels.tsa.arima.model.ARIMA.html)

    y_hat_diff = model.fit().forecast()[0]
    # predict the next timestep for the differenced data
    
    y_hat = inverse_difference(history, y_hat_diff, months_in_year)
    predictions.append(y_hat)
    # the corresponding prediction for the data (not differenced)

    history.append(test[i])
    # the corresponding actual data (not predicted)


# errors
residuals = [test[i]-predictions[i] for i in range(len(test))]
residuals = pd.DataFrame(residuals)
print(residuals.describe())

# plot
plt.figure()
plt.subplot(211)
residuals.hist(ax=plt.gca())
plt.subplot(212)
residuals.plot(kind='kde', ax=plt.gca())
plt.show()


### Part 6/7
- save

In [ ]:
# champagne sales example stripped & modified - part 6/7

# save model
# copy-paste the mean from the output of the previous cell
# this cell produces no output

from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA

# load data
df = read_csv('./data/dataset.csv', header=None, index_col=0, parse_dates=True).squeeze('columns')

# prepare data
X = df.values.astype('float32')

# difference data
months_in_year = 12
diff = difference(X, months_in_year)

# fit model
# replace the stars with your parameters from step 4
model = ARIMA(diff, order=(2,1,0))
model_fit = model.fit()

# this is the mean (bias) from the previous output
mean = -16.041581

# the previous output told us that the predictions were SYSTEMATICALLY
# a tiny bit larger than the actual values
# (larger because we subtracted the predictions from actual values and got a mean of -16)

# so we can just save that systematic "error" (bias) separately and take it into account
# in our future predictions

# save model (and the bias)
model_fit.save('./data/model.pkl')
np.save('./data/model_bias.npy', [mean])


### Part 7/7
- load
- predict

In [ ]:
# champagne sales example stripped & modified - part 7/7

# load and evaluate the model on the validation dataset
# note: the model is already produced and saved so
# loading and using it should be fast

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima.model import ARIMAResults
from sklearn.metrics import mean_squared_error

# load and prepare datasets
dataset = pd.read_csv('./data/dataset.csv', header=None, index_col=0, parse_dates=True).squeeze('columns')
X = dataset.values.astype('float32')

history = [x for x in X]
months_in_year = 12
validation = read_csv('./data/validation.csv', header=None, index_col=0, parse_dates=True).squeeze('columns')
y = validation.values.astype('float32')

# load model
model_fit = ARIMAResults.load('./data/model.pkl')
bias = np.load('./data/model_bias.npy')

# make the first prediction
predictions = list()
y_hat = float(list(model_fit.forecast())[0]) # this y_hat is a differenced series value
y_hat = bias + inverse_difference(history, y_hat, months_in_year) # this y_hat is an original series value
predictions.append(y_hat)
history.append(y[0])
print(f'Predicted={y_hat[0]:.3f}, Expected={y[0]:.3f}')

# rolling forecasts
for i in range(1, len(y)):
    months_in_year = 12
    diff = list(difference(history, months_in_year))
    model = ARIMA(diff, order=(2,1,0))
    model_fit = model.fit()
    y_hat = float(list(model_fit.forecast())[0])
    y_hat = bias + inverse_difference(history, y_hat, months_in_year)
    predictions.append(y_hat)

    # obs stands for observation
    # (the actual data)
    obs = y[i]
    history.append(obs)
    print(f'Predicted={y_hat[0]:.3f}, Expected={obs:.3f}')

# report performance
mse = mean_squared_error(y, predictions)
print('MSE: %.3f' % mse)
plt.plot(y)
plt.plot(predictions, color='red')
plt.show()


### P.S.
- We have skipped several parts of the original text. They were not really needed.

Below Harri's sktime ad hoc try from 2024

In [ ]:
import pandas as pd
import warnings

from sktime.forecasting.base import ForecastingHorizon
# we need to understand this better

from sktime.split import temporal_train_test_split
# this too

from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.naive import NaiveForecaster
# one can try different models

# forecaster = NaiveForecaster(strategy="last")
forecaster = AutoARIMA(
    sp=12, d=0, max_p=2, max_q=2, suppress_warnings=True
    # season parameter 12 hours is clearly not optimal for weather data
    # probaly 24 hours makes at least some difference (daily fluctuations should be seen in weather data)
) 

warnings.filterwarnings('ignore')
df = pd.read_csv('/home/varpha/dan/public/exrc_04/data/prob4_weather.csv')
df['time'] = pd.to_datetime(df['time'],unit='s')
df = df.set_index('time')

y = df['air_temperature_2m'].dropna().asfreq(freq='h')

y_train, y_test = temporal_train_test_split(y)
# the above needs to be done better
# in a manual fashion, like in the champagne thing
# (currently the returned y_train has a lot of NANs for some reason)

fh = ForecastingHorizon(
    y_test.index, is_relative=False
    # replace with something better
    # e.g. the next day after the test data
    # (instead of all the test data)
)
# print(fh)


forecaster.fit(y_train.dropna())
# one should not need to do dropna here

forecaster.predict(fh)
# print(df)

## On the *helmitaulu* cluster

Excerpt from `exrc_03/maths_notebooks/00_Python_Basics.ipynb` (helmitaulu uses the file-based approach):

>The style of programming that we will be undertaking in this subject is a `notebook` framework. This contrasts with the original style of python, which is more of a `file-based` (or synonymously, `module-based`) framework. 

>The ways these differ is that the `notebook` style:

>1. Uses cells and evaluates them one at a time.
>2. Produces output below the cell that called for it.
>3. Is (arguably) more common in scientific computing and data science.

>In contrast, the `file-based/module-based` style:

>1. Can keep code in multiple files that can then be accessed from one another.
>2. Produces all outputs to one location.
>3. Is very common in software engineering environments -- e.g. building apps and running websites.

There's a cluster system available for you at `helmitaulu.labranet.jamk.fi`. Requires vpn. You can log in from our jupyterhub command line (= Terminal) with `ssh helmitaulu` (labranet password). Essentially similar to csc puhti etc. Actually our jupyterhub runs on one of the helmitaulu nodes, but there are many more available (and mostly idle).

Some unix commands are available in our (jupyterhub) `public/exrc_04` directory.

### Steps required to run python stuff on helmitaulu

- load python module: `module load Python/3.9.5-GCCcore-10.3.0` (copy-pasted from the output of `module avail`; chosen by Harri based on trial and error)
- double check: `which python` or just try `python` and verify that it's 3.9 (Ctrl-D exits the python prompt)
- create a directory (python virtual environment in the workdir storage) for python libraries: `python -m venv ${WORKDIR}/dan` (dan stands for data analytics)
- activate the virtual environment: `source ${WORKDIR}/dan/bin/activate` (deactivating happens with just `deactivate`, and removing the virtual environment altogether is done by just removing the `${WORKDIR}/dan` directory with `rm -rf ${WORKDIR}/dan`)
- update the python package installer: `python -m pip install --upgrade pip`
- install the packages we need: `python -m pip install sktime pmdarima`
- downgrade numpy (sorry): `python -m pip install --upgrade numpy==1.26.4`
  - the above has to do with some missing C library (one could send a labranet helpdesk ticket about it)
- copy your dataset from jupyterhub to helmitaulu: at the jupyterhub terminal, do `scp /home/varpha/dan/public/exrc_04/data/prob4_weather.csv helmitaulu:` (nothing after the colon copies it to your home directory, but one could specify a destination there as well, like `scp /home/varpha/dan/public/exrc_04/data/prob4_weather.csv helmitaulu:path/to/my/directory`
- then edit (or `scp` from jupyterhub) a python script in a `.py` file, and run it with `srun python myscript.py` or (for more verbose) `srun -v python myscript.py`
- the `srun` command doesn't stream the output (at least by default), so you get all the output at once when the script has finished (some discussion about this [here](https://stackoverflow.com/questions/43397789/slurm-echo-output-file-so-it-prints-on-screen))
- in order to run your scripts with the `sbatch` command (and log out in between), you need an executable script such as this `myscript.sh`:
```bash
#!/usr/bin/sh
module load Python/3.9.5-GCCcore-10.3.0
source ${WORKDIR}/dan/bin/activate
cd ~
python test.py
```
- do e.g. `nano myscript.sh`, copy-paste the contents above, save & exit, do `chmod 700 myscript.sh` and finally `sbatch myscript.sh`; the output will come to a `slurm-XXX.out` file (where XXX is the job number).